# Update fits header and move fits files

## 모듈 설치 및 버전 확인

아래 셀을 실행하면 이 노트북을 실행하는데 필요한 모듈을 설치하고 파이썬 및 관련 모듈의 버전을 확인할 수 있습니다. 

In [16]:
#!pip install photutils==1.12

In [17]:
import importlib, sys, subprocess
packages = "ysfitsutilpy, version_information" # required modules
pkgs = packages.split(", ")
for pkg in pkgs :
    if not importlib.util.find_spec(pkg):
        # print(f"**** module {pkg} is not installed")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg, '-q'])
    else: 
        # print(f"**** module {pkg} is installed")
        pass

%load_ext version_information
import time
now = time.strftime("%Y-%m-%d %H:%M:%S (%Z = GMT%z)")
print(f"This notebook was generated at {now} ")

vv = %version_information {packages}
for i, pkg in enumerate(vv.packages):
    print(f"{i} {pkg[0]:10s} {pkg[1]:s}")

The version_information extension is already loaded. To reload it, use:
  %reload_ext version_information
This notebook was generated at 2024-12-14 00:09:13 (KST = GMT+0900) 
0 Python     3.12.7 64bit [GCC 11.2.0]
1 IPython    8.27.0
2 OS         Linux 6.8.0 50 generic x86_64 with glibc2.39
3 ysfitsutilpy 0.2
4 version_information 1.0.4


## import modules

우선 필필요요한  모모듈듈을 import합니다.

In [18]:
from pathlib import Path
import shutil
import os
import ysfitsutilpy as yfu

import _Python_utilities
import _astro_utilities

import warnings
warnings.filterwarnings('ignore')

## 디렉토리 환경 설정

새로 촬영한 파일들은 아래 폴더에 복사를 합니다. 처리해야 할 파일들이 들어 있는 폴더를 목록으로 만들어 둡니다.
이때 pathlib을 사용하면 운영체제에 관게 없이 파일 경로를 객체로 만들어 쉽게 다룰 수 있게 됩니다. 

In [19]:
#%%
verbose = False     
Owrite = False   
#######################################################
# Set directory variables.
#######################################################

BASEDIR = Path(r'S:\\')   # for windows
BASEDIR = Path("/mnt/Rdata/ASTRO_data")  # for ubuntu
# BASEDIR = Path("/Volumes/OBS_data")  # for mac OS
 
DOINGDIR = BASEDIR/ _astro_utilities.CCD_NEW_dir

DOINGDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(DOINGDIR))
print ("DOINGDIRs: ", format(DOINGDIRs))
print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))
#######################################################    
 

DOINGDIRs:  []
len(DOINGDIRs):  0


### 출력 변수 설정

In [20]:
fnameKEYs = ["OBJECT", "IMAGETYP", "FILTER", "DATE-OBS", 
            "EXPOSURE", "OPTIC", "CCDNAME", "CCD-TEMP", "XBINNING"]

## header update and move to

In [21]:
for DOINGDIR in DOINGDIRs[:] :
    DOINGDIR = Path(DOINGDIR)
    NEWUPDIR = DOINGDIR.parents[2] /_astro_utilities.CCD_NEWUP_dir
    if verbose == True : 
        print("DOINGDIR", DOINGDIR)
        print(f"Starting: {str(DOINGDIR.parts[-1])}")
    summary = yfu.make_summary(DOINGDIR/"*.fit*",
                                verify_fix=True,
                                ignore_missing_simple=True,
                                )
    if summary is not None : 
        try:
            if verbose == True : 
                print("summary: ", summary)
                print("len(summary)", len(summary))

            for _, row in summary.iterrows():
                if verbose == True : 
                    print (row["file"])   # 파일명 출력
                fpath = Path(row["file"])
            
                hdul = _astro_utilities.KevinFitsUpdater(fpath,
                                                # checkKEYs = ["OBJECT", "TELESCOP", "OPTIC", "CCDNAME", 'FILTER',
                                                #             #"GAIN", "EGAIN", "RDNOISE", 
                                                #             "PIXSCALE", "FOCALLEN", "APATURE", "CCD-TEMP",
                                                #             'XPIXSZ', 'YPIXSZ',
                                                #             "XBINNING", "YBINNING", "FLIPSTAT", "EXPTIME", "EXPOSURE"],
                                                # imgtype_update=True,
                                                # fil_update=False,
                                                verbose = False, 
                                                )
                if verbose == True :
                    print("hdul: ", hdul)
            
            shutil.move(str(DOINGDIR), str(NEWUPDIR / DOINGDIR.stem))
            if verbose == True :
                print(str(DOINGDIR), str(NEWUPDIR / DOINGDIR.stem))

        except Exception as err :
            print("X"*60)
            print(err)
            pass

    print(f"Starting: {str(NEWUPDIR.parts[-1])}") 
    summary = yfu.make_summary(NEWUPDIR/"*.fit*",
                                verify_fix=True,
                                ignore_missing_simple=True,
                                )
    if summary is not None : 
        if verbose == True :
            print("summary: ", summary)
            print("len(summary)", len(summary))
    
        for _, row in summary.iterrows():
            fpath = Path(row["file"])
            if verbose == True :
                print (f"starting {fpath.name}...")
            new_fname = ""
            suffix = ".fit"
            try:
                for KEY in fnameKEYs :
                    if KEY in ["OBJECT", "IMAGETYP", "FILTER", 
                        "OPTIC", "CCDNAME"] :
                        new_fname += str(row[KEY])+"_"
                    
                    if KEY == "DATE-OBS" : 
                        new_fname += row[KEY][:19].replace("T","-").replace(":","-")+"_"

                    if KEY == "EXPOSURE" : 
                        new_fname += str(int(row[KEY]))+"sec_"

                    if KEY == "CCD-TEMP" : 
                        try:
                            new_fname += str(int(row[KEY]))+"c_"
                        except:
                            new_fname += (row[KEY])+"c_"
                    if KEY == "XBINNING" : 
                        new_fname += str(row[KEY])+"bin"+suffix
                if verbose == True :
                    print(new_fname)                      
                new_folder = _astro_utilities.get_new_foldername_from_filename(new_fname)
                if verbose == True :
                    print("new_folder: ", new_folder)
                new_fpath =  BASEDIR /_astro_utilities.A3_CCD_obs_raw_dir / new_folder / new_fname
                if verbose == True :
                    print("new_fpath: ", new_fpath)

                if not new_fpath.parents[0].exists():
                    os.makedirs(f'{new_fpath.parents[0]}')
                    if verbose == True :
                        print(f'{new_fpath.parts[-2]} is created')  
            
                if new_fpath.exists() :
                    if verbose == True :
                        print(f'{new_fpath} is already exist')
                    duplicate_fpath = BASEDIR / _astro_utilities.CCD_duplicate_dir / new_fpath.name
                    if Owrite == False:
                        shutil.move(fpath, duplicate_fpath)
                        if verbose == True :
                            print(f'{fpath.parts[-1]} is move to duplicate folder...')
                    else :
                        shutil.move(str(fpath), str(new_fpath))
                        if verbose == True :
                            print(f"move {str(fpath.name)} to {str(new_fpath)}")
                else : 
                    shutil.move(str(fpath), str(new_fpath))
                    if verbose == True :
                        print(f"move {str(fpath.name)} to {str(new_fpath)}")
                
            except Exception as err:
                print("X"*30, f'\n{err}')
                #_Python_utilities.write_log(err_log_file, err)
                pass

In [23]:
#%%   
#############################################################################
#Check and delete empty folder....
#############################################################################
for i in range(4):
    DOINGDIR = ( BASEDIR/ _astro_utilities.CCD_NEWUP_dir)         
    DOINGDIRs = sorted(_Python_utilities.getFullnameListOfallsubDirs(DOINGDIR))
    if verbose == True :
        print ("DOINGDIRs: ", format(DOINGDIRs))
        print ("len(DOINGDIRs): ", format(len(DOINGDIRs)))

    for DOINGDIR in DOINGDIRs :    
        if len(os.listdir(str(DOINGDIR))) == 0 :
            shutil.rmtree(f"{str(DOINGDIR)}") # Delete..
            if verbose == True :
                print (f"rmtree {str(DOINGDIR)}")
        else : 
            fpaths = _Python_utilities.getFullnameListOfallFiles(str(DOINGDIR))
            if verbose == True :
                print("fpaths", fpaths)

            for fpath in fpaths[:]:
                if verbose == True :
                    print("fpath", fpath)

                if fpath[-4:].lower() in [".txt", "xisf", ".zip", ".png", ".log",
                                            "seal", "tiff", ".axy", "atch", "lved",
                                            "rdls", "xyls", "corr", "xosm", ".ini",
                                            ".wcs", ".csv"] \
                                        and os.path.isfile(fpath):
                    os.remove("{}".format(fpath))
                    if verbose == True :
                        print("{} is removed...".format(fpath)) 
